<a href="https://colab.research.google.com/github/nitinlodhi019/Machine-Learning/blob/main/Binary_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Upload Data

In [ ]:
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')

#Explore Dataset

In [ ]:
#shape and preview of data
print(train.shape)
train.sample(10)

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
303,LP001977,Male,Yes,1,Graduate,No,1625,1803.0,96.0,360.0,1.0,Urban,Y
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
32,LP001097,Male,No,1,Graduate,Yes,4692,0.0,106.0,360.0,1.0,Rural,N
45,LP001137,Female,No,0,Graduate,No,3410,0.0,88.0,NaN,1.0,Urban,Y
202,LP001682,Male,Yes,3+,Not Graduate,No,3992,0.0,NaN,180.0,1.0,Urban,N
214,LP001716,Male,Yes,0,Graduate,No,3173,3021.0,137.0,360.0,1.0,Urban,Y
270,LP001888,Female,No,0,Graduate,No,3237,0.0,30.0,360.0,1.0,Urban,Y
534,LP002731,Female,No,0,Not Graduate,Yes,18165,0.0,125.0,360.0,1.0,Urban,Y
247,LP001819,Male,Yes,1,Not Graduate,No,6608,0.0,137.0,180.0,1.0,Urban,Y
475,LP002527,Male,Yes,2,Graduate,Yes,16525,1014.0,150.0,360.0,1.0,Rural,Y


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [ ]:
#checking missing values in each column
train.apply(lambda x: sum(x.isnull()))

,0
Loan_ID,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14


#Fill missing values

**Gender**

In [ ]:
train['Gender'].value_counts()

,count
Gender,
Male,489
Female,112


In [ ]:
train.Gender = train.Gender.fillna('Male')

**Married**

In [ ]:
train['Married'].value_counts()

,count
Married,
Yes,398
No,213


In [ ]:
train.Married = train.Married.fillna('Yes')

**Dependents**

In [ ]:
train['Dependents'].value_counts()

,count
Dependents,
0,345
1,102
2,101
3+,51


In [ ]:
train.Dependents = train.Dependents.fillna('0')

**Self_Employed**

In [ ]:
train['Self_Employed'].value_counts()

,count
Self_Employed,
No,500
Yes,82


In [ ]:
train.Self_Employed = train.Self_Employed.fillna('No')

**LoanAmount**

In [ ]:
#train['LoanAmount'].fillna(train['LoanAmount'].mean(), inplace=True)
train.LoanAmount = train.LoanAmount.fillna(train.LoanAmount.mean())

**Loan_Amount_Term**

In [ ]:
train['Loan_Amount_Term'].value_counts()

,count
Loan_Amount_Term,
360.0,512
180.0,44
480.0,15
300.0,13
84.0,4
240.0,4
120.0,3
60.0,2
36.0,2


In [ ]:
train.Loan_Amount_Term = train.Loan_Amount_Term.fillna(360.0)

**Credit_History**

In [ ]:
train['Credit_History'].value_counts()

,count
Credit_History,
1.0,475
0.0,89


In [ ]:
train.Credit_History = train.Credit_History.fillna(1.0)

In [ ]:
#check missing values again
train.apply(lambda x: sum(x.isnull()), axis=0)

,0
Loan_ID,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,0
Loan_Amount_Term,0


In [ ]:
train.duplicated().sum()

np.int64(0)

In [ ]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y


In [ ]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.50,3812.5,5795.00,81000.0
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.00,1188.5,2297.25,41667.0
LoanAmount,614.0,146.412162,84.037468,9.0,100.25,129.0,164.75,700.0
Loan_Amount_Term,614.0,342.410423,64.428629,12.0,360.00,360.0,360.00,480.0
Credit_History,614.0,0.855049,0.352339,0.0,1.00,1.0,1.00,1.0


#Split Data

In [ ]:
#Splitting training data
X = train.drop(['Loan_ID', 'Loan_Status'], axis=1).values
y = train['Loan_Status'].values

In [ ]:
X

array([['Male', 'No', '0', ..., 360.0, 1.0, 'Urban'],
       ['Male', 'Yes', '1', ..., 360.0, 1.0, 'Rural'],
       ['Male', 'Yes', '0', ..., 360.0, 1.0, 'Urban'],
       ...,
       ['Male', 'Yes', '1', ..., 360.0, 1.0, 'Urban'],
       ['Male', 'Yes', '2', ..., 360.0, 1.0, 'Urban'],
       ['Female', 'No', '0', ..., 360.0, 0.0, 'Semiurban']], dtype=object)

In [ ]:
y

array(['Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'N', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N',
       'N', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'N',
       'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'N', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'N

In [ ]:
#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

array([['Male', 'Yes', '0', ..., 360.0, 1.0, 'Semiurban'],
       ['Male', 'Yes', '0', ..., 360.0, 1.0, 'Semiurban'],
       ['Male', 'Yes', '2', ..., 360.0, 1.0, 'Semiurban'],
       ...,
       ['Female', 'No', '0', ..., 360.0, 1.0, 'Urban'],
       ['Female', 'Yes', '0', ..., 240.0, 1.0, 'Semiurban'],
       ['Male', 'Yes', '0', ..., 360.0, 1.0, 'Urban']], dtype=object)

#Encoding

In [ ]:
labelencoder_X = LabelEncoder()

# Encoding the independent Variable
for i in range(0, 5):
    X_train[:,i] = labelencoder_X.fit_transform(X_train[:,i])

X_train[:,10] = labelencoder_X.fit_transform(X_train[:,10])

# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train)

In [ ]:
X_train

array([[1, 1, 0, ..., 360.0, 1.0, 1],
       [1, 1, 0, ..., 360.0, 1.0, 1],
       [1, 1, 2, ..., 360.0, 1.0, 1],
       ...,
       [0, 0, 0, ..., 360.0, 1.0, 2],
       [0, 1, 0, ..., 240.0, 1.0, 1],
       [1, 1, 0, ..., 360.0, 1.0, 2]], dtype=object)

In [ ]:
y_train

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,

In [ ]:
# Encoding the Independent Variable
labelencoder_X = LabelEncoder()
for i in range(0, 5):
    X_test[:,i] = labelencoder_X.fit_transform(X_test[:,i])
X_test[:,10] = labelencoder_X.fit_transform(X_test[:,10])

# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
y_test = labelencoder_y.fit_transform(y_test)

In [ ]:
X_test

array([[1, 1, 0, ..., 360.0, 1.0, 1],
       [1, 1, 0, ..., 360.0, 1.0, 1],
       [1, 1, 2, ..., 360.0, 1.0, 0],
       ...,
       [1, 1, 1, ..., 480.0, 1.0, 1],
       [1, 1, 2, ..., 360.0, 1.0, 2],
       [1, 1, 0, ..., 180.0, 1.0, 2]], dtype=object)

#Scaling

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

#Model Selection

##Logistic Regression

In [ ]:
# Fitting Logistic Regression to the Training set
classifier_lr = LogisticRegression(random_state = 1)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [ ]:
# Predicting the Test set results
y_pred = classifier_lr.predict(X_test)

# Measuring Accuracy
print('The accuracy of Logistic Regression is: ',accuracy_score(y_pred, y_test))

The accuracy of Logistic Regression is:  0.7886178861788617


In [ ]:
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[18, 25],
       [ 1, 79]])

##Polynomial LR

In [ ]:

def poly_lr(X,y,degree):

    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_trf = poly.fit_transform(X)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_trf,y)

    accuracy = np.mean(cross_val_score(clf,X_trf,y,scoring='accuracy',cv=10))
    print('Accuracy for degree',degree,'is',accuracy)

poly_lr(X_train,y_train,1)
poly_lr(X_train,y_train,2)
poly_lr(X_train,y_train,3)
poly_lr(X_train,y_train,4)

Accuracy for degree 1 is 0.8126938775510204
Accuracy for degree 2 is 0.7719591836734694
Accuracy for degree 3 is 0.7433877551020409
Accuracy for degree 4 is 0.7292244897959184


##KNN

In [ ]:
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
# Predicting the Test set results
y_pred = classifier_knn.predict(X_test)
print('The accuracy of KNN is: ',accuracy_score(y_pred, y_test))

The accuracy of KNN is:  0.7479674796747967


In [ ]:
# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[16, 27],
       [ 4, 76]])

##SVM

In [ ]:
classifier_svm = SVC(kernel = 'linear', random_state = 0)
classifier_svm.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [ ]:
# Predicting the Test set results
y_pred = classifier_svm.predict(X_test)
print('The accuracy of SVM is: ',accuracy_score(y_pred, y_test))

The accuracy of SVM is:  0.7886178861788617


##Naive Bayes

In [ ]:
classifier_nb = GaussianNB()
classifier_nb.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Predicting the Test set results
y_pred = classifier_nb.predict(X_test)
print('The accuracy of naive bayes is: ',accuracy_score(y_pred, y_test))

The accuracy of naive bayes is:  0.7642276422764228


##Decision tree

In [ ]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [ ]:
# Predicting the Test set results
y_pred = classifier_dt.predict(X_test)
print('The accuracy of Decision tree is: ',accuracy_score(y_pred, y_test))

The accuracy of Decision tree is:  0.6585365853658537


##Random Forest

In [ ]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [ ]:
# Predicting the Test set results
y_pred = classifier_rf.predict(X_test)
print('The accuracy of random forest is: ',accuracy_score(y_pred, y_test))

The accuracy of random forest is:  0.7154471544715447


**RESULT**:

* The accuracy of Logistic Regression is: 78.86%

* The accuracy of Polynomial Logistic Regression is:

Accuracy for degree 1 is 81.26%

Accuracy for degree 2 is 77.19%

Accuracy for degree 3 is 74.33%

Accuracy for degree 4 is 72.92%

* The accuracy of KNN is: 74.79%

* The accuracy of SVM is: 78.86%

* The accuracy of Naive Bayes is: 76.42%

* The accuracy of Decision Tree Classifier is: 65.85%

* The accuracy of Random Forest Classification is: 71.54%
